# USPTO AI TOPICS EXTRACTION
In this notebook we are using USPTO, United States Patent Office, patent's dataset to extract AI related patents. These patents are then integrated with other sources in which we have data on Organization and funding they have secired. Our goal is to see relationship between organization fundigs and patents. Organization with patents and funding may gave us some idea where more work in AI is headed. We also extract the topics from patent's abstract. In this way we can also see how what area a patent covers. This can lead us to the areas in AI which are being funded.

## Datasets
Datasets are downloaded from https://developer.uspto.gov/data. 
Data is available for public. Data contains all patents from 1970. Text only patent dataset is 3-6G in size.
1. Data filese are in tsv format
2. There are multiple data files that require integration to map an Organization to a patent

## Topics Extraction
We used pytextrank to extract top topics for each patent which has either AI or Artifical Intelligence in abstract of the patent.

### Challenges
We processed 2015 and onwards patents. We used threading to extract topics. Once topics are extracted we merged the patent with patent's assignee Organization data.


In [1]:
import pytextrank
import spacy
import pandas as pd
import numpy as np
from multiprocessing import  Pool

In [2]:
%%time
df = pd.read_csv('./datasets/patent.tsv',sep='\t')

/Users/ifti/anaconda3/envs/pytextrank/lib/python3.9/site-packages/IPython/core/magics/execution.py:1324: DtypeWarning: Columns (0,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code, glob, local_ns)


CPU times: user 51.3 s, sys: 8.24 s, total: 59.5 s
Wall time: 1min 2s


### Cleanup

- Patents date > 2015
- Abstract is not null
- Patent is not withdrawn

In [6]:
df_gt_2015 = df[df['date'].str[:4].astype(int) >= 2015]
df_gt_2015 = df_gt_2015[df_gt_2015['abstract'].notnull()]
df_gt_2015 = df_gt_2015[df_gt_2015['withdrawn']==0]

#### Filter out any patent which has no AI or Aritifical Intelligence word  

In [7]:
%%time
df_gt_2015_ai = df_gt_2015[df_gt_2015['abstract'].str.contains('(ai)|(artificial intelligence)', case=False, na=False)]

<timed exec>:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


CPU times: user 36.9 s, sys: 63.3 ms, total: 37 s
Wall time: 37 s


In [8]:
df_gt_2015_ai.shape

(806635, 11)

### Extract topics for each year using pytextrank

In [19]:
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")
def top_topics(text, max_items=5):
    # load a spaCy model, depending on language, scale, etc.
   
    doc = nlp(str(text))

    # examine the top-ranked phrases in the document
    top_ranked = []
    count = 0
    for phrase in doc._.phrases:
        count += 1
        if phrase.rank > .1 and count <= max_items:
#             print(phrase.text)
#             print(phrase.rank, phrase.count)
#             print(phrase.chunks)
            top_ranked.append(phrase.text)
        else:
            break

        
    return top_ranked

In [106]:
def add_topics(pdf, year, prefix="df_"):

    df = pdf[pdf['date'].str[:4].astype(int) == year]
    print(f'in topics df size {len(df)}')
    df['top_topics'] = df.apply(lambda row: top_topics(row['abstract'], 3), axis=1)
    print(f'creating csv df_{year}.csv')
    df.to_csv(f'{prefix}{year}.csv', columns=['id','date','country','title','top_topics'])

### Parallel processing 

We processed last five years of data. Each year would take 5-6 hrs. We parallel run topics for each extraction and then merged the results back. 

In [101]:
import threading
num_threads = 10
chunk_size = round(len(df_gt_2015_ai)/num_threads)
chunk_size = 1
curr_idx = 0

for year in range(2017, 2022):
    print(f'processing {year}')
    threading.Thread(target=add_topics, args=(df_gt_2015_ai, year, )).start()    

2017
2018
2019
2020
2021
in topics df size 117365in topics df size 122101

in topics df size 135008in topics df size 134474

in topics df size 64629


#### Merged the results back to one dataframe.

In [120]:
cols=['id','date','country','title','top_topics']
df_2017 = pd.read_csv('./df_2017.csv', usecols=cols)
df_2018 = pd.read_csv('./df_2018.csv', usecols=cols)
df_2019 = pd.read_csv('./df_2019.csv', usecols=cols)
df_2020 = pd.read_csv('./df_2020.csv', usecols=cols)
df_2021 = pd.read_csv('./df_2021.csv', usecols=cols)

/Users/ifti/anaconda3/envs/pytextrank/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [135]:
df_gt_2017 = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021], ignore_index=True)

In [136]:
df_gt_2017

,id,date,country,title,top_topics
0,9532496,2017-01-03,US,Dynamic supplemental downforce control system ...,"['planting operations', 'the actual gauge whee..."
1,9532500,2017-01-03,US,Digger for rhizomes and the like,"['adhered soil', 'soil', 'rhizomes']"
2,9532501,2017-01-03,US,Plant conveyor and method for harvesting plants,"['bias', 'plants']"
3,9532503,2017-01-03,US,Monofilament line holder for grass trimmers,"['garden equipment', 'longitudinal bores']"
4,9532504,2017-01-03,US,Control arrangement and method for controlling...,"['image representing container', 'retrieved im..."
...,...,...,...,...,...
573572,RE48604,2021-06-22,US,Scanner module and image scanning apparatus em...,"['light', 'thermal expansion', 'convex deforma..."
573573,RE48606,2021-06-22,US,Generation of HARQ-ACK information and power c...,"['DL Scheduling Assignments', 'multiple transm..."
573574,RE48607,2021-06-29,US,"Thickener composition, thickened nutritive pro...","['said chelating agent', 'food product', 'asco..."
573575,RE48609,2021-06-29,US,Three-dimensional printing apparatus,"['display units', 'projected patterns', 'the d..."


## Merge with Location and assignee

In [157]:
df_assignee = pd.read_csv('./datasets/assignee.tsv',sep='\t')
df_patent_assignee = pd.read_csv('./datasets/patent_assignee.tsv', sep='\t')
df_location =  pd.read_csv('./datasets/location.tsv', sep='\t')
df_patent_assignee = df_patent_assignee.merge(df_assignee, left_on='assignee_id', right_on='id', suffixes=('_x', '_y'))
df_patent_assignee = df_patent_assignee.merge(df_location, left_on='location_id', right_on='id', suffixes=('', '_location'))

/Users/ifti/anaconda3/envs/pytextrank/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [158]:
df_gt_2017_assignee_location = df_gt_2017.merge(df_patent_assignee, left_on='id', right_on='patent_id', suffixes=('','_x'))

In [138]:
print(df_gt_2017_assignee_location.shape)
df_gt_2017_assignee_location

(277952, 22)


,id,date,country,title,top_topics,patent_id,assignee_id,location_id,id_x,type,...,organization,id_location,city,state,country_x,latitude,longitude,county,state_fips,county_fips
0,PP27524,2017-01-03,US,Ilex plant named ‘Screen Play’,"['color', 'plants', 'Ilex plant']",PP27524,3b86ee5d-49e0-444b-800a-67eb3c0b4f2c,aa885578-cb8f-11eb-9615-121df0c29c1e,3b86ee5d-49e0-444b-800a-67eb3c0b4f2c,2.0,...,ITSAUL PLANTS LLC,aa885578-cb8f-11eb-9615-121df0c29c1e,Alpharetta,GA,US,34.0674,-84.26930,Fulton,13.0,13121.0
1,PP27527,2017-01-03,US,Echeveria plant named ‘Cubic Frost’,"['landscape plants', 'container plants', 'lila...",PP27527,ed25dd8e-2465-47f2-b1e7-00d223089af4,e5bc0ebf-cb8e-11eb-9615-121df0c29c1e,ed25dd8e-2465-47f2-b1e7-00d223089af4,2.0,...,Altman Plants,e5bc0ebf-cb8e-11eb-9615-121df0c29c1e,Vista,CA,US,33.2000,-117.24300,San Diego,6.0,6073.0
2,PP27529,2017-01-03,US,Heuchera plant named ‘Van Gogh’,"['dark red flowering', 'red veins', 'small lea...",PP27529,ec677381-057a-4c19-98fa-902347528481,5bf3ccd3-cb8e-11eb-9615-121df0c29c1e,ec677381-057a-4c19-98fa-902347528481,5.0,...,NaN,5bf3ccd3-cb8e-11eb-9615-121df0c29c1e,Hantay,NaN,FR,50.5378,2.86741,NaN,NaN,NaN
3,PP27536,2017-01-10,US,Perennial peanut ‘PP-1’,"['dark green leaf color', 'low maintenance', '...",PP27536,51a6da5b-4755-4e2f-a282-159552a45a42,3ca33312-cb8e-11eb-9615-121df0c29c1e,51a6da5b-4755-4e2f-a282-159552a45a42,2.0,...,"Georgia State University Research Foundation, ...",3ca33312-cb8e-11eb-9615-121df0c29c1e,Athens,GA,US,33.9598,-83.37640,Clarke,13.0,13059.0
4,PP27537,2017-01-10,US,Pelargonium plant named ‘PEPZ0005’,"['large single florets', 'continuous color', '...",PP27537,37d3b9fd-1d06-4d7a-a5dd-1f9334bbd292,ff643dfd-cb90-11eb-9615-121df0c29c1e,37d3b9fd-1d06-4d7a-a5dd-1f9334bbd292,3.0,...,SYNGENTA PARTICIPATIONS AG,ff643dfd-cb90-11eb-9615-121df0c29c1e,Basel,NaN,CH,47.5545,7.59025,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277947,RE48602,2021-06-22,US,Gun safety device,"['firearm', 'chamber', 'a muzzle device']",RE48602,b9f2fc1a-c4c9-4f2f-a367-836b11aa3436,68c20688-cb8e-11eb-9615-121df0c29c1e,b9f2fc1a-c4c9-4f2f-a367-836b11aa3436,4.0,...,NaN,68c20688-cb8e-11eb-9615-121df0c29c1e,St. Albans,ME,US,44.9101,-69.41000,NaN,23.0,NaN
277948,RE48604,2021-06-22,US,Scanner module and image scanning apparatus em...,"['light', 'thermal expansion', 'convex deforma...",RE48604,802e1b9f-b4e7-4295-867a-6db6a2ab117c,d0f1f2d6-cb8f-11eb-9615-121df0c29c1e,802e1b9f-b4e7-4295-867a-6db6a2ab117c,2.0,...,Hewlett Packard Enterprise Development LP,d0f1f2d6-cb8f-11eb-9615-121df0c29c1e,Spring,TX,US,30.0799,-95.41730,Harris,48.0,48201.0
277949,RE48606,2021-06-22,US,Generation of HARQ-ACK information and power c...,"['DL Scheduling Assignments', 'multiple transm...",RE48606,39719b1e-eb6d-414e-b351-7430992a204b,ff9566ad-cb8f-11eb-9615-121df0c29c1e,39719b1e-eb6d-414e-b351-7430992a204b,3.0,...,"Samsung Electronics Co., Ltd.",ff9566ad-cb8f-11eb-9615-121df0c29c1e,Suwon-si,NaN,KR,37.2875,127.01000,NaN,NaN,NaN
277950,RE48607,2021-06-29,US,"Thickener composition, thickened nutritive pro...","['said chelating agent', 'food product', 'asco...",RE48607,5faab021-bfdc-4326-9623-efee6829a4d1,609a4d6f-09bd-11ec-893a-12de62d610b1,5faab021-bfdc-4326-9623-efee6829a4d1,3.0,...,"Kent Precision Foods Group, Inc.",609a4d6f-09bd-11ec-893a-12de62d610b1,Missouri,NaN,MO,NaN,NaN,NaN,NaN,NaN


In [159]:
df_gt_2017_assignee_location.drop(columns=['state_fips', 'county_fips','latitude','longitude','id_x','name_first','name_last','type','patent_id','location_id'], inplace=True)

In [141]:
headers = ['id','number','title','date','country','top_topics','assignee_id','location_id', 'organization','state','country_x']
# df_gt_2021_assignee_location[:100].to_csv('patents_topics_gt_2021_100.csv', columns=headers, index=False)

In [142]:
df_org = df_gt_2017_assignee_location[['assignee_id','organization']]

In [143]:
df_org = df_org.drop_duplicates().dropna()
df_org.shape

(45750, 2)

In [144]:
df_org.to_csv('patent_org_2017.csv',index=False)

## Merge with VenturBeat Processed 

In [147]:
df_vb_patent = pd.read_csv('df_vb_patent.csv')

In [148]:
df_vb_patent

,Date,Header,Abstract,Clean_Funding,Topics,Company,assignee_id,organization
0,2021-12-05,Sense raises $50M to bolster recruitment effor...,Recruiting is a top concern for enterprises in...,50000000.0,"['job candidate', 'Candidates', 'candidates']",Sense raises $50M to bolster recruitment effor...,b0c02ce9-3761-4171-a6ce-6b1e8c95fda2,Sensera Inc.
1,2021-12-02,Digital Insulin Management Company Hygieia Clo...,"LIVONIA, Mich.–(BUSINESS WIRE)–December 2, 202...",17000000.0,"['insulin management', 'insulin therapy', 'ins...",Digital Insulin Management Company Hygieia Clo...,c3795cf3-5530-4b77-9f57-1642a79b029b,Digital Doc LLC
2,2021-11-22,IoT anomaly detection startup Shield-IoT lands...,"Shield-IoT , a provider of AI-driven security ...",7400000.0,"['IoT networks', 'Solomon', 'Udi Solomon']",IoT anomaly detection startup Shield-IoT lands...,ef871799-fcda-4ccd-91a1-39cf473a811c,"IoT Holdings, Inc."
3,2021-11-18,Engineering software startup nTopology lands $65M,"NTopology , a company that develops software u...",65000000.0,"['new product designs', 'different design soft...",Engineering software startup nTopology lands $65M,f00b159e-a652-4959-af9c-268737b4692b,Engineered Supply L.L.C.
4,2021-11-17,Stellar Cyber raises $38M to provide 360-degre...,"Stellar Cyber, the innovator of Open XDR, has ...",38000000.0,"['endpoint security solutions', 'better securi...",Stellar Cyber raises $38M to provide 360-degre...,5e528075-10a1-418a-ad7d-f8866e17cb2c,Stellar Biome Inc.
...,...,...,...,...,...,...,...,...
436,2014-11-02,Jonathan Rothberg's Butterfly Network has rais...,Noted biosciences entrepreneur Jonathan Rothbe...,100000000.0,"['4Combinator companies', 'Jonathan Rothberg',...",Jonathan Rothberg's Butterfly Network has rais...,d3c92046-a18b-4441-bd3f-f020e0a9b0cb,Jonathan LEVY
437,2014-07-21,Mobile messaging platform Tango hires game pub...,Tango wants everyone to know it is serious ab...,25000000.0,"['Game developers', 'game developers', 'mobile...",Mobile messaging platform Tango hires game pub...,ce001d86-16e7-4fb6-8b53-191e60313d46,"Mobile Tech, Inc."
438,2013-06-20,Tempo AI secures $10M to advance its productiv...,"Tempo AI , a startup focused on increasing pro...",10000000.0,"['Eniac Ventures', 'Qualcomm Ventures', 'Horiz...",Tempo AI secures $10M to advance its productiv...,bd509861-c985-4fb1-a9f6-eed4bbee38fd,Tempo Manufacturing LLC
439,2012-06-28,Online ad player Rocket Fuel gains $50M fundin...,"Rocket Fuel, a four-year-old ad technology fi...",50000000.0,"['Nokia Growth Capital', 'advertising opportun...",Online ad player Rocket Fuel gains $50M fundin...,37a30269-4c5e-4b8e-b262-63421a6c2abe,"Online Development, Inc."


In [169]:
df_gt_2017_patent_vb = df_gt_2017_assignee_location.merge(df_vb_patent[['assignee_id','Clean_Funding']], how='left', left_on='assignee_id', right_on='assignee_id', suffixes=('_x', '_y'))

## Merge SBIR Funding Data

In [162]:
df_sb_patent = pd.read_csv('df_sb_patent.csv')

In [167]:
df_sb_patent['Award Amount'] = df_sb_patent['Award Amount'].str.replace(',', '').astype(float)

In [168]:
df_sb_patent

,Company,organization,assignee_id,Award Amount,Socially and Economically Disadvantaged,Women Owned,State
0,"Aura Intelligent Systems, Inc.","Aura Intelligent Systems, Inc.",b3b07262-1aa1-4cff-b14c-4ecbf9995a86,256000.0,N,Y,MA
1,"Chiral Photonics, Inc.","Chiral Photonics, Inc.",1255db6b-aa99-43bf-b63b-bb1f3f18b18f,1125745.0,N,N,NJ
2,Farad Power Inc,"Farad Power, Inc",42cf33c3-6c26-4fa8-a326-e580c019ce46,132383.0,N,N,CA
3,"Gate Scientific, Inc.","Gate Scientific, Inc.",46bb2e8b-7bd4-45eb-862d-55d097e5d1fa,999656.0,N,N,CA
4,"H3D, Inc.","H3D, Inc.",203dee44-5faa-4bae-b2ab-b12e492e9177,200000.0,N,N,MI
...,...,...,...,...,...,...,...
987,Plasma Tech Corp.,"Plasma Tech Holdings, LLC",ee79c702-d68d-48d7-86c1-f59744dd4afd,41415.0,N,N,IL
988,"Synergetics, Inc.","Synergetics, Inc.",80c21baa-31ec-4bf9-ae87-962881913fcb,371845.0,N,N,NC
989,Environmental Protection,"Environmental Protection Administration, R.O.C.",ab221a9c-1281-4a8e-ac7c-478f3dd18f73,50000.0,N,N,CA
990,General Technology Inc,Generalplus Technology Inc.,0de653f4-d162-4cfc-890b-503c9623e346,49607.0,N,N,VA


In [171]:
cols = ['assignee_id', 'Award Amount', 'Socially and Economically Disadvantaged', 'Women Owned','State']
df_gt_2017_patent_vb_sb = df_gt_2017_patent_vb.merge(df_sb_patent[cols], how='left', left_on='assignee_id', right_on='assignee_id', suffixes=('_x', '_y'))

In [176]:
# df_gt_2017_patent_vb_sb[df_gt_2017_patent_vb_sb['Award Amount']>0  df_gt_2017_patent_vb_sb['Clean_Funding']>0]
df_gt_2017_patent_vb_sb[df_gt_2017_patent_vb_sb['Clean_Funding']>0]

,id,date,country,title,top_topics,assignee_id,organization,id_location,city,state,country_x,county,Clean_Funding,Award Amount,Socially and Economically Disadvantaged,Women Owned,State
141,PP28310,2017-08-22,US,Fragaria plant named ‘MALGA’,"['medium consistency', 'Plants', 'plant']",5d27d219-2d88-4218-a5a6-51732392a3ca,New Fruits SAS,e78a0c14-cb8e-11eb-9615-121df0c29c1e,NaN,NaN,IT,NaN,11000000.0,NaN,NaN,NaN,NaN
385,9946856,2018-04-17,US,On-demand application permissions,['associated consents'],4b661a6e-291b-42ab-a44e-7c1156e8c8e4,Google LLC,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Mountain View,CA,US,Santa Clara,3370000.0,NaN,NaN,NaN,NaN
387,9946867,2018-04-17,US,Input mirroring,"['user input', 'first', 'The first component']",e62cea34-5174-4cfd-a59d-7ebea8b78dca,"Amazon Technologies, Inc.",fea686f6-09bd-11ec-893a-12de62d610b1,Seattle,WA,US,King,100000000.0,NaN,NaN,NaN,NaN
397,9946895,2018-04-17,US,Data obfuscation,"['keys', 'Sensitive data']",e62cea34-5174-4cfd-a59d-7ebea8b78dca,"Amazon Technologies, Inc.",fea686f6-09bd-11ec-893a-12de62d610b1,Seattle,WA,US,King,100000000.0,NaN,NaN,NaN,NaN
398,9946899,2018-04-17,US,Active ASIC intrusion shield,"['traces', 'digital logic components']",4b661a6e-291b-42ab-a44e-7c1156e8c8e4,Google LLC,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Mountain View,CA,US,Santa Clara,3370000.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277620,RE47992,2020-05-12,US,Network-assisted fabric pairing,"['fabric credentials', 'a joining device', 'th...",4b661a6e-291b-42ab-a44e-7c1156e8c8e4,Google LLC,ff4c2272-cb8e-11eb-9615-121df0c29c1e,Mountain View,CA,US,Santa Clara,3370000.0,NaN,NaN,NaN,NaN
277796,PP32895,2021-03-16,US,Kiwi plant named ‘Red Queen’,"['fruit shape', 'mid-October', 'unpatented cul...",477dd3dc-6d50-408c-ae3e-e835bcc4504b,Red Kiwi S.r.l.,c0eca240-cb8e-11eb-9615-121df0c29c1e,Riolo Terme,NaN,IT,NaN,38000000.0,NaN,NaN,NaN,NaN
277820,PP33000,2021-04-27,US,Cannabis plant named ‘OG CITRON’,"['flower samples', 'myrcene', 'limonene']",78e6b22a-58bf-4af1-b17d-31aab54381cb,"Biotech Institute, LLC",98be0390-cb8f-11eb-9615-121df0c29c1e,Westlake Village,CA,US,Los Angeles,400000000.0,NaN,NaN,NaN,NaN
277911,RE48484,2021-03-23,US,Signal processing apparatus and methods,"['computer communications', 'local computers',...",71c27494-8259-4e7f-96d9-12847ede59f8,Personalized Media Communications LLC,fcb936ec-cb8e-11eb-9615-121df0c29c1e,Sugar Land,TX,US,Fort Bend,17500000.0,NaN,NaN,NaN,NaN


In [177]:
# df_gt_2017_patent_vb_sb['funding_amount'] = df_gt_2017_patent_vb_sb[]
df_gt_2017_patent_vb_sb['funding_amount'] = df_gt_2017_patent_vb_sb.Clean_Funding.combine_first(df_gt_2017_patent_vb_sb['Award Amount'])


In [180]:
df_gt_2017_patent_vb_sb_funded = df_gt_2017_patent_vb_sb[df_gt_2017_patent_vb_sb.funding_amount>0]

In [181]:
df_gt_2017_patent_vb_sb_funded.to_csv('df_gt_2017_patent_vb_sb_funded.csv', index=False)